In [ ]:
#!pip install openai

In [ ]:
# Ustawienia OpenAI 
APIKey = ''

import openai
openai.api_key = APIKey

In [ ]:
from openai import OpenAI
client = OpenAI(
    api_key=APIKey,
)

In [ ]:
import pandas as pd
import random
number_of_products = 50

# Lista przykładowych produktów
producers = ['Samsung', 'LG', 'Bosch', 'Whirlpool', 'Electrolux', 'Amica']
types = ['Lodówka', 'Kuchenka gazowa', 'Zmywarka', 'Płyta indukcyjna', 'Płyta elektryczna']
models = [f'Model-{i}' for i in range(1, number_of_products +1)]

# Generowanie danych dla DataFrame (naszej bazy produktów)

data = {
    'Producent': [random.choice(producers) for _ in range(number_of_products)],
    'Model': random.sample(models, number_of_products),
    'Cena': [random.randint(1000, 5000) for _ in range(number_of_products)],
    'Typ urządzenia': [random.choice(types) for _ in range(number_of_products)]
}

# Utwórz DataFrame
df = pd.DataFrame(data)


In [ ]:
df

## Konwersacja

In [ ]:
query = "Znajdź Lodówkę"

In [ ]:
system_prompt = """Jesteś inteligentną wyszukiwarką produktów kuchennych. Pomagasz wyszukać sprzęt do kuchni spełniający wymagania klienta. 
Dopytujesz o szczegóły sprzętu, który użytkownik chce kupić: Rodzaj sprzętu, producenta i maksymalną cenę.
"""

In [ ]:
messages=[
            {"role": "developer", "content": system_prompt},        
            {"role": "user", "content": query}
]

In [ ]:
messages

In [ ]:
function_desc = [
    {
        "type": "function",                 
        "name": "product_search",           
        "description": "Wyszukiwanie sprzętu kuchennego. Zawsze wypełniaj wszystkie pola.",
        "strict": False,
        "parameters": {                     
            "type": "object",
            "properties": {
                "Producent": {
                    "type": "string",
                    "description": "Producent urządzenia: Samsung', 'LG', 'Bosch', 'Whirlpool', 'Electrolux', 'Amica'"
                },
                "Cena": {
                    "type": "number",
                    "description": "Maksymalna cena urządzenia (liczba)"
                },
                "Typ urządzenia": {
                    "type": "string",
                    "description": "Rodzaj sprzętu: Lodówka, Zmywarka, Płyta elektryczna, płyta indukcyjna, Kuchenka gazowa'"
                }
            },
            "required": ["Producent", "Cena",  "Typ urządzenia"]
        }
    }
]


In [ ]:
response = client.responses.create(
    model="gpt-5-mini",
    tools=function_desc,
    input=messages
)

In [ ]:
response

In [ ]:
import json

message = response  # cały obiekt odpowiedzi
args_dict = {}
# Przechodzimy przez wszystkie elementy w output (mogą być reasoning, tool call, text)
for item in message.output:

    # 1. Funkcja została wywołana
    if item.type == "function_call":
        print(f"Function: {item.name}")
        args_dict = json.loads(item.arguments)
        print("Znalezione wymagania:", args_dict)
    
    # 2. Zwykła odpowiedź asystenta (wiadomość)
    elif getattr(item, "type", None) == "message":
        # item to ResponseOutputMessage
        for part in item.content:
            # part to ResponseOutputText / inne typy contentu
            if getattr(part, "type", None) == "output_text":
                has_text = True
                print("Odpowiedź:", part.text)




In [ ]:
args_dict

In [ ]:
filtered_df = df[
    (df["Producent"] == args_dict["Producent"]) &
    (df["Cena"] <= args_dict["Cena"]) &
    (df["Typ urządzenia"] == args_dict["Typ urządzenia"])
]

In [ ]:
filtered_df

In [ ]:
results = []

for _, row in filtered_df.iterrows():
    results.append(f"Marka:{row['Producent']}, model:{row['Model']}, cena:{row['Cena']}, Typ:{row['Typ urządzenia']}")

equipment = "\n".join(results)
print(equipment)

In [ ]:
messages=[
            {"role": "developer", "content": "Jesteś inteligentną wyszukiwrką produktów kuchennych. Masz poinformować użytkowniak o znalexionym sprzęcie, który spełnai jego wymagania"},
            {"role": "developer", "content": "znaleziony sprzęt to:"+equipment},
    
            {"role": "user", "content": query}
]

In [ ]:
response = client.responses.create(
    model="gpt-5-mini",
    input=messages
)
print(response.output_text.strip())